# **Modèle de référence (dummy)**

## **Importation des bibliothèques**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pour les traîtements sur les variables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modèle de ML utilisé
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier

# Pour gridsearch
from sklearn import model_selection

from sklearn import metrics

## **Importation du dataset**

In [2]:
dataset_a_utiliser = "data_clean.xlsx"

data = pd.read_excel(dataset_a_utiliser)
print('Importation terminée')

Importation terminée


## **Split & Scale**

In [3]:
######################
# Choix des features #
######################

features_list = [
                 'BuildingType',
                 'PrimaryPropertyType',
                 'Neighborhood',
                 'YearBuilt',
                 'NumberofBuildings',
                 'NumberofFloors',
                 'PropertyGFAParking',
                 'PropertyGFABuilding(s)'
                ]

y_column = 'SiteEnergyUseWN_kBtu'
#y_column = 'TotalGHGEmissions'

###############################
# Split du dataset par X et y #
###############################

X = data.loc[:,features_list] # On ajoute nos features dans le X
y = data.loc[:,y_column] # On ajoute ce qu'on veut prédire dans le y


#####################################################################
# Scaling des données quantitatives et qualitatives (si existantes) #
# Je fais le scale ici car il y a des 'YearBuilt' uniques (ex 1939) #
# et du coup je ne pourrai pas faire un transform sur X_test        #
#####################################################################

categorical_features = [
                        'BuildingType',
                        'PrimaryPropertyType',
                        'Neighborhood',
                        'YearBuilt'
                       ]

numeric_features = [
                    'NumberofBuildings',
                    'NumberofFloors',
                    'PropertyGFAParking',
                    'PropertyGFABuilding(s)'
                   ]

numeric_transformer = StandardScaler()

categorical_transformer = OneHotEncoder()

feature_encoder = ColumnTransformer(
            transformers=[
                ('cat', categorical_transformer, categorical_features),    
                ('num', numeric_transformer, numeric_features)
                ]
            )
X = feature_encoder.fit_transform(X)


#####################################
# Split du dataset en train et test #
#####################################

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=0
                                                    )
                                                    #stratify=y) # : Stratification du sample. 
                                                    # Il y aura alors la même proportion
                                                    # de catégories dans les sets de test et train.
                                                    # Utile surtout s'il y a peu de lignes dans le dataset.
                                                    # Peut causer une erreur si certaines catégories n'ont pas assez de représentants


print("Split & Scale : OK")

Split & Scale : OK


## **Entraînement du modèle *DummyRegressor()***

In [4]:
modele = DummyRegressor() # Stratégie par défaut : mean
modele.fit(X_train, y_train) # Étape d'entraînement
print("Entraînement : OK")

Entraînement : OK


## **Score du modèle**

In [5]:
print(f"Le score R² du modèle est de {modele.score(X_train, y_train):.3f} (train) et {modele.score(X_test, y_test):.3f} (test).")

Le score R² du modèle est de 0.000 (train) et -0.013 (test).


## **GridSearch - DummyRegressor()**

Test des meilleurs hyperparamètres. Dans le cadre d'une régression linéraire simple, pas grand chose à tester.

In [6]:
###############################################
# Paramétrage de GridSearchCV et entraînement #
###############################################

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'strategy':['mean', 'median', 'quantile', 'constant'], 'constant':range(int(data[y_column].min()), int(data[y_column].max()), int(data[y_column].max()/3)), 'quantile':[0.0, 0.25, 0.5, 0.75, 1.0]}
# Pour constant j'ai fait un range de min y à max y, avec un pas de max/3

# Déterminer le score qu'on veut optimiser
score = 'r2'

# Je le sors ici car je vais réutiliser cette valeur plus tard
cv = 5

grid = model_selection.GridSearchCV(
    DummyRegressor(), # On indique le modèle à tester
    param_grid,     # hyperparamètres à tester
    cv=cv,           # nombre de folds de validation croisée
    scoring=score   # score à optimiser
)

# Optimiser ce modèle sur le jeu d'entraînement
grid.fit(X_train, y_train)

###########################
# Affichage des résultats #
###########################

# Afficher le(s) hyperparamètre(s) optimaux
print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement : {grid.best_params_}\n")

# Afficher les performances correspondantes
# print('Résultats pour chaque fold :')
# for i in range(cv):
#    print(f"Fold n°{i+1} :", eval(f"grid.cv_results_['split{i}_test_score']"), f"(Pour les paramètres : {grid.cv_results_['params']})")

#print("\nRésultats de la validation croisée :")
#for mean, std, params in zip(
#        grid.cv_results_['mean_test_score'], # score moyen
#        grid.cv_results_['std_test_score'],  # écart-type du score
#        grid.cv_results_['params']           # valeur de l'hyperparamètre
#    ):

#    print(f"{score} (moyen) : {mean:.04f} (+/-{std*2:.04f}) pour {params}")

###################################################################
# Prédiction sur le jeu de test avec les hyperparamètres optimaux #
###################################################################

# GridSearchCV a automatiquement ré-entraîné le meilleur modèle sur l’intégralité du jeu d’entraînement.
y_pred = grid.predict(X_test)
print(f"\nScore sur le jeu de test (avec paramètres optimaux): {metrics.r2_score(y_test, y_pred):.3f}")



Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement : {'constant': 58114, 'quantile': 0.75, 'strategy': 'quantile'}


Score sur le jeu de test (avec paramètres optimaux): -0.011


## **Entraînement du modèle *DummyClassifier***

### **Au préalable je dois refaire le split and scale, puisque y change.**

In [7]:
######################
# Choix des features #
######################

features_list = [
                 'BuildingType',
                 'PrimaryPropertyType',
                 'Neighborhood',
                 'YearBuilt',
                 'NumberofBuildings',
                 'NumberofFloors',
                 'PropertyGFAParking',
                 'PropertyGFABuilding(s)'
                ]

y_column = 'SiteEnergyUseWN_kBtu_cat'
#y_column = 'TotalGHGEmissions_cat'

###############################
# Split du dataset par X et y #
###############################

X = data.loc[:,features_list] # On ajoute nos features dans le X
y = data.loc[:,y_column] # On ajoute ce qu'on veut prédire dans le y


#####################################################################
# Scaling des données quantitatives et qualitatives (si existantes) #
# Je fais le scale ici car il y a des 'YearBuilt' uniques (ex 1939) #
# et du coup je ne pourrai pas faire un transform sur X_test        #
#####################################################################

categorical_features = [
                        'BuildingType',
                        'PrimaryPropertyType',
                        'Neighborhood',
                        'YearBuilt'
                       ]

numeric_features = [
                    'NumberofBuildings',
                    'NumberofFloors',
                    'PropertyGFAParking',
                    'PropertyGFABuilding(s)'
                   ]

numeric_transformer = StandardScaler()

categorical_transformer = OneHotEncoder()

feature_encoder = ColumnTransformer(
            transformers=[
                ('cat', categorical_transformer, categorical_features),    
                ('num', numeric_transformer, numeric_features)
                ]
            )
X = feature_encoder.fit_transform(X)


#####################################
# Split du dataset en train et test #
#####################################

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=0
                                                    )
                                                    #stratify=y) # : Stratification du sample. 
                                                    # Il y aura alors la même proportion
                                                    # de catégories dans les sets de test et train.
                                                    # Utile surtout s'il y a peu de lignes dans le dataset.
                                                    # Peut causer une erreur si certaines catégories n'ont pas assez de représentants


print("Split & Scale : OK")

Split & Scale : OK


### **Entraînement**

In [8]:
modele = DummyClassifier() # Stratégie par défaut : prior
modele.fit(X_train, y_train) # Étape d'entraînement
print("Entraînement : OK")

Entraînement : OK


## **Score du modèle**

In [9]:
print(f"Le score du modèle est de {modele.score(X_train, y_train):.3f} (train) et {modele.score(X_test, y_test):.3f} (test).")

Le score du modèle est de 0.207 (train) et 0.170 (test).


## **GridSearch - DummyClassifier()**

Test des meilleurs hyperparamètres.

In [10]:
###############################################
# Paramétrage de GridSearchCV et entraînement #
###############################################

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'strategy':['most_frequent', 'prior', 'stratified', 'uniform']}
# Pour constant j'ai fait un range de min y à max y, avec un pas de max/3

# Déterminer le score qu'on veut optimiser
score = 'accuracy'

# Je le sors ici car je vais réutiliser cette valeur plus tard
cv = 5

grid = model_selection.GridSearchCV(
    DummyClassifier(), # On indique le modèle à tester
    param_grid,     # hyperparamètres à tester
    cv=cv,           # nombre de folds de validation croisée
    scoring=score   # score à optimiser
)

# Optimiser ce modèle sur le jeu d'entraînement
grid.fit(X_train, y_train)

###########################
# Affichage des résultats #
###########################

# Afficher le(s) hyperparamètre(s) optimaux
print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement : {grid.best_params_}\n")

# Afficher les performances correspondantes
print('Résultats pour chaque fold :')
for i in range(cv):
    print(f"Fold n°{i+1} :", eval(f"grid.cv_results_['split{i}_test_score']"), f"(Pour les paramètres : {grid.cv_results_['params']})")

print("\nRésultats de la validation croisée :")
for mean, std, params in zip(
        grid.cv_results_['mean_test_score'], # score moyen
        grid.cv_results_['std_test_score'],  # écart-type du score
        grid.cv_results_['params']           # valeur de l'hyperparamètre
    ):

    print(f"{score} (moyen) : {mean:.04f} (+/-{std*2:.04f}) pour {params}")

###################################################################
# Prédiction sur le jeu de test avec les hyperparamètres optimaux #
###################################################################

# GridSearchCV a automatiquement ré-entraîné le meilleur modèle sur l’intégralité du jeu d’entraînement.
y_pred = grid.predict(X_test)
print(f"\nScore sur le jeu de test (avec paramètres optimaux): {metrics.accuracy_score(y_test, y_pred):.3f}")



Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement : {'strategy': 'most_frequent'}

Résultats pour chaque fold :
Fold n°1 : [0.20491803 0.20491803 0.15163934 0.22131148] (Pour les paramètres : [{'strategy': 'most_frequent'}, {'strategy': 'prior'}, {'strategy': 'stratified'}, {'strategy': 'uniform'}])
Fold n°2 : [0.20491803 0.20491803 0.17622951 0.19672131] (Pour les paramètres : [{'strategy': 'most_frequent'}, {'strategy': 'prior'}, {'strategy': 'stratified'}, {'strategy': 'uniform'}])
Fold n°3 : [0.20901639 0.20901639 0.18852459 0.20081967] (Pour les paramètres : [{'strategy': 'most_frequent'}, {'strategy': 'prior'}, {'strategy': 'stratified'}, {'strategy': 'uniform'}])
Fold n°4 : [0.20901639 0.20901639 0.19262295 0.16393443] (Pour les paramètres : [{'strategy': 'most_frequent'}, {'strategy': 'prior'}, {'strategy': 'stratified'}, {'strategy': 'uniform'}])
Fold n°5 : [0.20576132 0.20576132 0.18106996 0.22222222] (Pour les paramètres : [{'strategy': 'most_frequent'}, {'strategy': '